In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
data = pd.read_stata("BES2015_W3_v4_9.dta")

In [3]:
england_data = data[data["country"] == "England"].reset_index()

scottish_data = data[data["country"] == "Scotland"].reset_index()

wales_data = data[data["country"] == "Wales"].reset_index()

WalScot_data = data[data["country"].isin(["Wales", "Scotland"])].reset_index()

print(england_data.shape[0], "наблюдений про Англию")
print(scottish_data.shape[0], "наблюдений про Шотландию")
print(wales_data.shape[0], "наблюдений про Уэльс")
print(WalScot_data.shape[0], "наблюдений про Шотландию с Уэльсом")

19494 наблюдений про Англию
5232 наблюдений про Шотландию
3113 наблюдений про Уэльс
8345 наблюдений про Шотландию с Уэльсом


# Шотландцы

In [4]:
scottish_data = scottish_data[["britishness", "scottishness", "scotReferendumVote", "partyId"]]

In [5]:
cleanup_nums = {"scottishness": {"Not at all Scottish": 1,  "Very strongly Scottish": 7, "Don't know" : 0},
                "britishness" : {"Not at all British" : 1, "Very strongly British" : 7, "Don't know" : 0},
                "scotReferendumVote" :{"I voted 'No' (Scotland should not be an independent country)": 0, "I voted 'Yes' (Scotland should be an independent country)" : 1, "Don't know" : np.nan}
               }

scottish_data.replace(cleanup_nums, inplace=True)

In [6]:
scottish_data["iv_ident_crisis"] = np.nan
for row in range(len(scottish_data)):
    if scottish_data.loc[row, "britishness"] in [0,1,2,3] or scottish_data.loc[row, "scottishness"] == 0:
        scottish_data.loc[row, "iv_ident_crisis"] = 1
    else:
        scottish_data.loc[row, "iv_ident_crisis"] = 0

In [7]:
scottish_data["dv_nationalism"] = np.nan
for row in range(len(scottish_data)):
    if scottish_data.loc[row, "scotReferendumVote"] == 1 or scottish_data.loc[row, "partyId"] == "Scottish National Party (SNP)":
        scottish_data.loc[row, "dv_nationalism"] = 1
    else:
        scottish_data.loc[row, "dv_nationalism"] = 0

In [8]:
import statsmodels.api as sm
logit_model=sm.Logit(scottish_data["dv_nationalism"], scottish_data["iv_ident_crisis"])
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.624386
         Iterations 6
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.097    
Dependent Variable: dv_nationalism   AIC:              6535.5796
Date:               2018-09-20 20:49 BIC:              6542.1421
No. Observations:   5232             Log-Likelihood:   -3266.8  
Df Model:           0                LL-Null:          -3615.7  
Df Residuals:       5231             LLR p-value:      nan      
Converged:          1.0000           Scale:            1.0000   
No. Iterations:     6.0000                                      
----------------------------------------------------------------
                    Coef.  Std.Err.    z    P>|z|  [0.025 0.975]
----------------------------------------------------------------
iv_ident_crisis     2.3575   0.1185 19.9000 0.0000 2.1253 2.5897



Объяснение результатов:<br>
P-value при независимой переменной статитистически значимый (меньше 5% уровня значимости), поэтому связь между независимой и зависимой переменной действительно существует.<br>
Направление связи определяем по знаку перед коэффициентом - знак +, а это значит, что направление связи прямое: чем больше независимая, тем больше и вероятность того, что будет значение зависимой переменной. В данном случае, при наличие кризиса идентичности, будет скорее всего и национализм.<br>
**НЕ ПОКАЗЫВАЙ НИКОМУ СВОЙ ПСЕВДО R КВАДРАТ**

## Англичане

In [9]:
england_data = england_data[["britishness", "englishness", "partyId", "happyScotIndepResultNo", "scotIndepGoodEng", "scotFairShare", "walesFairShare", "englandGovern"]]

In [10]:
cleanup_nums = {"englishness":  {"Not at all English": 1, "Very strongly English": 7, "Don't know" : 0},
                "britishness" : {"Not at all British" : 1, "Very strongly British" : 7, "Don't know" : 0},
                "happyScotIndepResultNo" : {"Extremely happy" : 10, "Extremely disappointed" : 0, "Don't know" : 0}
               }

england_data.replace(cleanup_nums, inplace=True)

In [11]:
england_data["iv_ident_crisis"] = np.nan
for row in range(len(england_data)):
    if england_data.loc[row, "britishness"] == 0 or england_data.loc[row, "englishness"] in [0,1,2,3]:
        england_data.loc[row, "iv_ident_crisis"] = 1
    else:
        england_data.loc[row, "iv_ident_crisis"] = 0

In [12]:
england_data["dv_nationalism"] = np.nan
for row in range(len(england_data)):
    if england_data.loc[row, "partyId"] == "United Kingdom Independence Party (UKIP)" or england_data.loc[row, "partyId"] == "British National Party (BNP)" or (england_data.loc[row, "englishness"] in [6,7] and england_data.loc[row, "britishness"] in [6,7]) or england_data.loc[row, "happyScotIndepResultNo"] in [6,7,8,9,10] or england_data.loc[row, "scotFairShare"] in ["Much more than its fair share", "A little more than its fair share"] or england_data.loc[row, "walesFairShare"] in ["Much more than its fair share", "A little more than its fair share"] or england_data.loc[row, "englandGovern"] in ["By the UK Parliament, but with only English MPs having a vote", "By a new, separate parliament for England, but with England remaining part of the UK", "By a new, separate parliament for England, and with England becoming independent from the rest of the UK"]:
        england_data.loc[row, "dv_nationalism"] = 1
    else:
        england_data.loc[row, "dv_nationalism"] = 0

In [13]:
import statsmodels.api as sm
logit_model_england=sm.Logit(england_data["dv_nationalism"], england_data["iv_ident_crisis"])
result_england=logit_model_england.fit()
print(result_england.summary2())

Optimization terminated successfully.
         Current function value: 0.688759
         Iterations 4
                         Results: Logit
Model:              Logit            Pseudo R-squared: -1.056    
Dependent Variable: dv_nationalism   AIC:              26855.3452
Date:               2018-09-20 20:51 BIC:              26863.2231
No. Observations:   19494            Log-Likelihood:   -13427.   
Df Model:           0                LL-Null:          -6531.8   
Df Residuals:       19493            LLR p-value:      nan       
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     4.0000                                       
------------------------------------------------------------------
                 Coef.   Std.Err.     z     P>|z|   [0.025  0.975]
------------------------------------------------------------------
iv_ident_crisis  0.7447    0.0589  12.6421  0.0000  0.6293  0.8602



Объяснение результатов:<br>
P-value при независимой переменной статитистически значимый (меньше 5% уровня значимости), поэтому связь между независимой и зависимой переменной действительно существует.<br>
Направление связи определяем по знаку перед коэффициентом - знак +, а это значит, что направление связи прямое: чем больше независимая, тем больше и вероятность того, что будет значение зависимой переменной. В данном случае, при наличие кризиса идентичности, будет скорее всего и национализм.<br>
**НЕ ПОКАЗЫВАЙ НИКОМУ СВОЙ ПСЕВДО R КВАДРАТ**

# Wales

In [14]:
wales_data = wales_data[["britishness", "welshness", "partyId", "walesFairShare", "engFairShare"]]

In [15]:
wales_data["iv_ident_crisis"] = np.nan
for row in range(len(wales_data)):
    if wales_data.loc[row, "britishness"] == "Don't know" or wales_data.loc[row, "welshness"] in ["Don't know", "Not at all Welsh",2,3]:
        wales_data.loc[row, "iv_ident_crisis"] = 1
    else:
        wales_data.loc[row, "iv_ident_crisis"] = 0

In [16]:
wales_data["dv_nationalism"] = np.nan
for row in range(len(wales_data)):
    if wales_data.loc[row, "partyId"] == "Plaid Cymru" or wales_data.loc[row, "welshness"] == "Very strongly Welsh" or wales_data.loc[row, "walesFairShare"] in ["Much less than its fair share", "A little less than its fair share"] or wales_data.loc[row, "engFairShare"] in ["Much more than its fair share", "A little more than its fair share"]:
        wales_data.loc[row, "dv_nationalism"] = 1
    else:
        wales_data.loc[row, "dv_nationalism"] = 0

In [17]:
import statsmodels.api as sm
logit_model_wales=sm.Logit(wales_data["dv_nationalism"], wales_data["iv_ident_crisis"])
result_wales=logit_model_wales.fit()
print(result_wales.summary2())

Optimization terminated successfully.
         Current function value: 0.692173
         Iterations 4
                         Results: Logit
Model:              Logit            Pseudo R-squared: -0.278   
Dependent Variable: dv_nationalism   AIC:              4311.4698
Date:               2018-09-20 20:51 BIC:              4317.5131
No. Observations:   3113             Log-Likelihood:   -2154.7  
Df Model:           0                LL-Null:          -1686.0  
Df Residuals:       3112             LLR p-value:      nan      
Converged:          1.0000           Scale:            1.0000   
No. Iterations:     4.0000                                      
-----------------------------------------------------------------
                 Coef.   Std.Err.    z     P>|z|   [0.025  0.975]
-----------------------------------------------------------------
iv_ident_crisis  0.2059    0.0838  2.4561  0.0140  0.0416  0.3703



Объяснение результатов:<br>
P-value при независимой переменной статитистически значимый (меньше 5% уровня значимости), поэтому связь между независимой и зависимой переменной действительно существует.<br>
Направление связи определяем по знаку перед коэффициентом - знак +, а это значит, что направление связи прямое: чем больше независимая, тем больше и вероятность того, что будет значение зависимой переменной. В данном случае, при наличие кризиса идентичности, будет скорее всего и национализм.<br>
**НЕ ПОКАЗЫВАЙ НИКОМУ СВОЙ ПСЕВДО R КВАДРАТ**

# Wales + Scotland

In [18]:
WalScot_data = WalScot_data[["britishness", "scottishness", "scotReferendumVote", "partyId", "welshness", "country", "walesFairShare", "engFairShare"]]

In [19]:
WalScot_data["iv_ident_crisis"] = np.nan
for row in range(len(WalScot_data)):
    if WalScot_data.loc[row, "britishness"] == "Don't know" or (WalScot_data.loc[row, "welshness"] in ["Don't know", "Not at all Welsh",2,3] and WalScot_data.loc[row, "country"] == "Wales") or (WalScot_data.loc[row, "scottishness"] in ["Don't know", "Not at all Scottish",2,3] and WalScot_data.loc[row, "country"] == "Scotland"):
        WalScot_data.loc[row, "iv_ident_crisis"] = 1
    else:
        WalScot_data.loc[row, "iv_ident_crisis"] = 0

In [20]:
WalScot_data["dv_nationalism"] = np.nan
for row in range(len(WalScot_data)):
    if WalScot_data.loc[row, "scotReferendumVote"] == "I voted 'Yes' (Scotland should be an independent country)" or WalScot_data.loc[row, "partyId"] in ["Scottish National Party (SNP)", "Plaid Cymru"] or WalScot_data.loc[row, "walesFairShare"] in ["Much less than its fair share", "A little less than its fair share"] or WalScot_data.loc[row, "engFairShare"] in ["Much more than its fair share", "A little more than its fair share"] or WalScot_data.loc[row, "walesFairShare"] in ["Much less than its fair share", "A little less than its fair share"]:
        WalScot_data.loc[row, "dv_nationalism"] = 1
    else:
        WalScot_data.loc[row, "dv_nationalism"] = 0

In [21]:
import statsmodels.api as sm
logit_model_WalScot=sm.Logit(WalScot_data["dv_nationalism"], WalScot_data["iv_ident_crisis"])
result_WalScot=logit_model_WalScot.fit()
print(result_WalScot.summary2())

Optimization terminated successfully.
         Current function value: 0.693048
         Iterations 3
                         Results: Logit
Model:              Logit            Pseudo R-squared: -0.104    
Dependent Variable: dv_nationalism   AIC:              11568.9650
Date:               2018-09-20 20:53 BIC:              11575.9944
No. Observations:   8345             Log-Likelihood:   -5783.5   
Df Model:           0                LL-Null:          -5236.9   
Df Residuals:       8344             LLR p-value:      nan       
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     3.0000                                       
------------------------------------------------------------------
                 Coef.   Std.Err.    z     P>|z|    [0.025  0.975]
------------------------------------------------------------------
iv_ident_crisis  0.0791    0.0614  1.2885  0.1976  -0.0412  0.1995



Объяснение результатов:<br>
P-value при независимой переменной статитистически значимый (меньше 5% уровня значимости), поэтому связь между независимой и зависимой переменной действительно существует.<br>
Направление связи определяем по знаку перед коэффициентом - знак +, а это значит, что направление связи прямое: чем больше независимая, тем больше и вероятность того, что будет значение зависимой переменной. В данном случае, при наличие кризиса идентичности, будет скорее всего и национализм.<br>
**НЕ ПОКАЗЫВАЙ НИКОМУ СВОЙ ПСЕВДО R КВАДРАТ**